In [1]:
%pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
def json_to_matrix_df(data, label: str = ""):
    # Convert each metric's dictionary to a pandas Series
    series_dict = {}
    for metric, stats in data.items():
        if metric == "chunkCount":
            # store as int data type
            series_dict[metric] = pd.Series(stats).astype("int32")
        elif metric in ["timeToFirstChunk", "totalTime"]:
            series_dict[metric] = (pd.Series(stats) / 1_000).round(2)
        else:
            series_dict[metric] = pd.Series(stats).round(2)
    
    # Combine all series into a DataFrame
    df = pd.DataFrame(series_dict).T

    # add label to df
    if label:
        df.index.name = label
    
    return df

In [4]:
import json

def compare_and_display_data(source1: str, source2: str):
    with open(source1) as file:
        d1 = json.loads(file.read())

    with open(source2) as file:
        d2 = json.loads(file.read())
    
    keys = set(d1.keys()).union(set(d2.keys()))
    print(keys)
    for key in keys:
        print("-" * 5, key, "-" * 5)

        if key in d1:
            display(json_to_matrix_df(d1[key], label=source1))

        if key in d2:
            display(json_to_matrix_df(d2[key], label=source2))

### Ollama vs vLLM

1. **concurrent, cold-start:** 10 concurrent requests when container is _NOT_ up 
2. **concurrent, warm:** 10 concurrent requests after container is already up **(warm)**
3.  **sequential, cold-start:** 10 _sequential_ requests, when the container is NOT up

In [5]:
compare_and_display_data("./ollama/stats.json", "./vllm/stats.json")

{'concurrent-10-same-prompt-cold-start', 'concurrent-10-same-prompt-warm', 'sequential-same-prompt', 'concurrent-10-sp-OLLAMA_NUM_PARALLEL-10'}
----- concurrent-10-same-prompt-cold-start -----


,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,10.23,25.44,16.49,18.04,18.53,25.38,25.43,25.44
totalTime,17.76,32.60,23.89,25.33,25.86,32.01,32.48,32.60
chunkCount,303.00,444.00,361.00,356.00,377.00,419.00,439.00,444.00
averageChunksPerSecond,10.97,21.00,15.87,13.76,20.42,20.84,20.97,21.00


,min,max,average,p50,p75,p95,p99,p100
./vllm/stats.json,,,,,,,,
timeToFirstChunk,66.34,66.65,66.62,66.65,66.65,66.65,66.65,66.65
totalTime,74.20,79.33,76.77,76.85,77.47,78.67,79.20,79.33
chunkCount,257.00,435.00,345.00,348.00,370.00,410.00,430.00,435.00
averageChunksPerSecond,3.46,5.48,4.48,4.53,4.79,5.22,5.43,5.48


----- concurrent-10-same-prompt-warm -----


,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,2.45,16.46,7.56,8.47,9.49,15.18,16.21,16.46
totalTime,7.82,21.56,14.30,14.99,18.00,21.08,21.46,21.56
chunkCount,251.00,425.00,323.00,323.00,346.00,392.00,418.00,425.00
averageChunksPerSecond,13.91,34.97,25.00,21.90,33.51,34.75,34.93,34.97


...

----- sequential-same-prompt -----


,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,0.47,10.20,1.47,0.50,0.52,5.84,9.33,10.20
totalTime,4.51,14.49,6.12,5.29,5.49,10.56,13.71,14.49
chunkCount,313.00,404.00,360.00,363.00,375.00,398.00,402.00,404.00
averageChunksPerSecond,23.25,70.90,65.33,69.92,70.20,70.72,70.86,70.90


,min,max,average,p50,p75,p95,p99,p100
./vllm/stats.json,,,,,,,,
timeToFirstChunk,0.45,63.89,6.84,0.50,0.54,35.4,58.19,63.89
totalTime,8.47,74.39,16.39,10.15,11.08,46.0,68.71,74.39
chunkCount,291.00,393.00,348.00,352.00,378.00,391.0,392.00,393.00
averageChunksPerSecond,5.14,34.80,31.69,34.68,34.70,34.8,34.80,34.80


----- concurrent-10-sp-OLLAMA_NUM_PARALLEL-10 -----


,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,0.49,0.54,0.52,0.53,0.53,0.54,0.54,0.54
totalTime,12.35,15.57,13.79,14.08,14.16,15.27,15.51,15.57
chunkCount,301.00,440.00,349.00,353.00,355.00,420.00,436.00,440.00
averageChunksPerSecond,24.22,28.26,25.25,25.05,25.13,27.53,28.12,28.26


## Ollama vs vLLM: batched concurrent requests: 1, 2, ... 10

In [6]:
import json
import pandas as pd
from typing import Dict, Any
from pathlib import Path

def batched_json_to_df(file_path: str, field: str) -> pd.DataFrame:
    """
    Convert a nested JSON file containing performance metrics into a 2D pandas DataFrame.
    
    Parameters:
    -----------
    file_path : str
        Path to the JSON file containing nested performance metrics
    field : str
        The metric field to extract (e.g., "timeToFirstChunk", "totalTime")
        
    Returns:
    --------
    pd.DataFrame
        A DataFrame where:
        - Index: Number of concurrent requests (1, 2, 3, ...)
        - Columns: Statistics (min, max, average, p50, ...)
        - Values: The corresponding metric values
        
    Example:
    --------
    >>> df = json_to_df("metrics.json", "timeToFirstChunk")
    >>> print(df)
              min       max   average       p50       p75       p95       p99      p100
    1  542.062209 542.06221 542.06221 542.06221 542.06221 542.06221 542.06221 542.06221
    2  369.981792 644.45421 507.21800 507.21800 575.83610 630.73059 641.70948 644.45421
    """
    
    # Validate and read JSON file
    try:
        with open(file_path, 'r') as f:
            data: Dict[str, Any] = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find file: {file_path}")
    except json.JSONDecodeError:
        raise ValueError(f"Invalid JSON format in file: {file_path}")
    
    # Validate field exists
    if not any(field in entry for entry in data.values()):
        raise ValueError(f"Field '{field}' not found in data")
    
    # Define stat columns in desired order
    stat_columns = ['min', 'max', 'average', 'p50', 'p75', 'p95', 'p99', 'p100']
    
    # Extract data into dictionary for DataFrame
    df_data = {}
    for req_num, metrics in data.items():
        if field in metrics:
            df_data[int(req_num)] = {
                stat: metrics[field][stat]
                for stat in stat_columns
                if stat in metrics[field]
            }
    
    # Create DataFrame and sort index
    df = pd.DataFrame.from_dict(df_data, orient='index')
    df.sort_index(inplace=True)
    
    return df

## Ollama vs vLLM: batched concurrent requests: 1, 2, ... 10

In [7]:
print("----- time to first chunk (seconds) -----")
print("ollma:")
ollama = (batched_json_to_df("./ollama/batched-concurrent-1-10-same-prompt-warm/stats.json", "timeToFirstChunk") / 1_000).round(3)
display(ollama)
print("vLLM:")
vllm = (batched_json_to_df("./vllm/batched-concurrent-1-10-same-prompt-warm/stats.json", "timeToFirstChunk") / 1_000).round(3)
display

----- time to first chunk (seconds) -----
ollma:


,min,max,average,p50,p75,p95,p99,p100
1,0.635,0.635,0.635,0.635,0.635,0.635,0.635,0.635
2,0.344,0.499,0.421,0.421,0.460,0.492,0.498,0.499
3,0.351,0.504,0.411,0.379,0.442,0.492,0.502,0.504
4,0.431,0.505,0.450,0.432,0.451,0.494,0.503,0.505
5,0.356,7.518,1.806,0.386,0.387,6.091,7.232,7.518
6,0.351,8.977,2.851,0.499,4.837,8.303,8.842,8.977
7,0.344,8.467,3.588,0.518,7.437,8.178,8.409,8.467
8,0.350,8.906,4.153,3.745,7.533,8.765,8.878,8.906
9,0.346,15.154,5.660,8.038,8.457,12.854,14.694,15.154
10,0.354,15.298,6.207,7.267,8.128,14.974,15.233,15.298


vLLM:


,min,max,average,p50,p75,p95,p99,p100
1,0.542,0.542,0.542,0.542,0.542,0.542,0.542,0.542
2,0.370,0.644,0.507,0.507,0.576,0.631,0.642,0.644
3,0.359,0.489,0.407,0.374,0.432,0.478,0.487,0.489
4,0.354,0.481,0.392,0.366,0.395,0.464,0.477,0.481
5,0.361,0.496,0.422,0.383,0.495,0.496,0.496,0.496
6,0.355,0.464,0.401,0.375,0.440,0.463,0.464,0.464
7,0.364,0.502,0.428,0.380,0.500,0.501,0.502,0.502
8,0.352,0.466,0.402,0.387,0.406,0.465,0.465,0.466
9,0.362,0.483,0.412,0.384,0.474,0.480,0.482,0.483
10,0.365,0.516,0.416,0.395,0.397,0.515,0.516,0.516


In [9]:
print("----- tokens per second -----")
print("ollma:")
display(batched_json_to_df("./ollama/batched-concurrent-1-10-same-prompt-warm/stats.json", "averageChunksPerSecond").round(3))
print("vLLM:")
display(batched_json_to_df("./vllm/batched-concurrent-1-10-same-prompt-warm/stats.json", "averageChunksPerSecond").round(3))

----- tokens per second -----
ollma:


,min,max,average,p50,p75,p95,p99,p100
1,63.699,63.699,63.699,63.699,63.699,63.699,63.699,63.699
2,56.467,57.224,56.845,56.845,57.034,57.186,57.216,57.224
3,46.263,49.890,48.441,49.170,49.530,49.818,49.876,49.890
4,43.625,48.815,45.467,44.714,45.932,48.238,48.700,48.815
5,29.540,45.959,41.116,43.731,44.031,45.573,45.882,45.959
6,22.425,44.575,37.473,43.114,44.155,44.527,44.565,44.575
7,21.534,44.525,34.586,42.979,43.664,44.465,44.513,44.525
8,20.817,44.443,33.445,34.047,44.003,44.372,44.429,44.443
9,13.152,44.479,30.754,23.996,44.221,44.421,44.467,44.479
10,17.414,44.130,30.245,23.933,43.767,44.025,44.109,44.130


vLLM:


,min,max,average,p50,p75,p95,p99,p100
1,34.704,34.704,34.704,34.704,34.704,34.704,34.704,34.704
2,33.822,34.389,34.105,34.105,34.247,34.360,34.383,34.389
3,33.909,34.120,34.027,34.054,34.087,34.113,34.119,34.120
4,33.006,34.307,33.728,33.799,34.045,34.255,34.297,34.307
5,33.026,34.023,33.392,33.220,33.477,33.914,34.001,34.023
6,32.810,33.801,33.373,33.410,33.563,33.745,33.790,33.801
7,32.744,33.539,33.100,33.002,33.277,33.525,33.536,33.539
8,31.979,33.389,32.953,33.113,33.225,33.358,33.383,33.389
9,32.264,33.304,32.750,32.607,33.046,33.205,33.284,33.304
10,32.288,32.787,32.543,32.533,32.607,32.760,32.782,32.787
